# Для синтетических датасетов

## Интегральный MSE

In [6]:
from scipy.integrate import quad
import numpy as np

def integral_mse(g_hat, g_true, a=-2, b=8, epsabs=1e-5):
    """
    Вычисляет интегральный MSE: (1/(b-a)) * ∫[a,b] (ĝ(x) - g(x))² dx
    
    Параметры:
    g_hat : callable
        Оценённая регрессия (принимает float, возвращает float)
    g_true : callable
        Истинная регрессия (принимает float, возвращает float)
    a, b : float, optional
        Границы интегрирования (по умолчанию [-2, 8])
    epsabs : float, optional
        Абсолютная точность интегрирования (по умолчанию 1e-5)
    
    Возвращает:
    float
        Значение интегрального MSE
    """
    integrand = lambda x: (g_hat(x) - g_true(x)) ** 2
    integral, _ = quad(integrand, a, b, epsabs=epsabs)
    return integral / (b - a)

## Интегральный MAE

In [7]:
def integral_mae(g_hat, g_true, a=-2, b=8, epsabs=1e-5):
    """
    Вычисляет интегральный MAE: (1/(b-a)) * ∫[a,b] |ĝ(x) - g(x)| dx
    
    Параметры и возвращает:
    То же, что и integral_mse
    """
    integrand = lambda x: abs(g_hat(x) - g_true(x))
    integral, _ = quad(integrand, a, b, epsabs=epsabs)
    return integral / (b - a)

## Максимальная ошибка (MaxErr)

In [8]:
from scipy.optimize import minimize_scalar

def max_error(g_hat, g_true, a=-2, b=8, xatol=1e-5):
    """
    Вычисляет максимальную ошибку: max_{x ∈ [a,b]} |ĝ(x) - g(x)|
    
    Параметры:
    xatol : float, optional
        Абсолютная точность поиска максимума (по умолчанию 1e-5)
    
    Возвращает:
    float
        Значение максимальной ошибки
    """
    # Функция для минимизации: отрицание модуля ошибки
    objective = lambda x: -abs(g_hat(x) - g_true(x))
    
    # Надёжный поиск максимума на интервале
    result = minimize_scalar(
        objective,
        bounds=(a, b),
        method='bounded',
        options={'xatol': xatol}
    )
    
    # Гарантируем корректность даже при сбое оптимизации
    if not result.success:
        # Резерв: оценка на сетке из 100 точек
        x_grid = np.linspace(a, b, 100)
        errors = [abs(g_hat(x) - g_true(x)) for x in x_grid]
        return max(errors)
    
    return -result.fun  # Возвращаем положительное значение ошибки

# Пример

In [ ]:
# Пример для смеси (ваша аналитическая функция)
def g_true(x):
    A = np.exp(-(x - 1)**2 / 2)
    B = np.exp(-(x - 3)**2 / 18)
    return 3 - (24 * A) / (12 * A + B)

# Пример оценки: полином 3-й степени
def g_hat_example(x):
    return 0.5 + 0.8*x - 0.2*x**2 + 0.01*x**3

# Вычисление метрик
mse_val = integral_mse(g_hat_example, g_true)
mae_val = integral_mae(g_hat_example, g_true)
max_val = max_error(g_hat_example, g_true)

print(f"MSE_int = {mse_val:.6f}")
print(f"MAE_int = {mae_val:.6f}")
print(f"MaxErr  = {max_val:.6f}")

MSE_int = 4.402197
MAE_int = 1.674982
MaxErr  = 3.779997


# Метрики для реального датасета

In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

def cv_metrics(X, y, n_splits=5, random_state=42):
    """
    Вычисляет 5-fold CV RMSE и MAE для любого метода регрессии
    
    Параметры:
    X : array-like, shape (n_samples, n_features)
        Признаки
    y : array-like, shape (n_samples,)
        Целевая переменная
    n_splits : int, optional (default=5)
        Число фолдов
    random_state : int, optional (default=42)
        Для воспроизводимости перемешивания
    
    Возвращает:
    dict
        {'rmse': среднее RMSE по фолдам, 'mae': среднее MAE по фолдам}
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rmse_scores = []
    mae_scores = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # ===============================
        # МЕСТО ДЛЯ ВАШЕЙ МОДЕЛИ
        # ===============================
        # 1. Обучение модели:
        #    model = ваш_метод_обучения(X_train, y_train)
        #
        # 2. Предсказание на тестовом фолде:
        #    y_pred = ваш_метод_предсказания(model, X_test)
        #
        # Пример для линейной регрессии:
        #   from sklearn.linear_model import LinearRegression
        #   model = LinearRegression().fit(X_train, y_train)
        #   y_pred = model.predict(X_test)
        #
        # ВАЖНО: y_pred должен быть 1D-массивом длины len(y_test)
        # ===============================
        
        # Заглушка для теста (замените своим кодом!)
        y_pred = np.mean(y_train) * np.ones_like(y_test)  # <-- УДАЛИТЬ ЭТУ СТРОКУ ПОСЛЕ ВСТАВКИ СВОЕЙ МОДЕЛИ
        
        # Вычисление метрик для текущего фолда
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        
        rmse_scores.append(rmse)
        mae_scores.append(mae)
    
    return {
        'rmse': np.mean(rmse_scores),
        'mae': np.mean(mae_scores)
    }

In [1]:
b = [5, 9]
for i in range(3, 14):
    print(i)
    b.append(b[-1] + b[-2] * 4)
    print(b[-1])

3
29
4
65
5
181
6
441
7
1165
8
2929
9
7589
10
19305
11
49661
12
126881
13
325525
